# Справочники Cross Web
Пример ноутбука для работы со справочниками Cross Web.

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter 

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from pathlib import Path
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

mnet = mscore.MediascopeApiNetwork()
cats = cwc.CrossWebCats()

# Доступные атрибуты для параметров

При формировании задания можно указать следующие параметры:

- **filters** - Фильтры
- **slices** - Срезы
- **statistics** - Статистики
- **scales** - Шкалы, если заданы статистики со шкалами (drfd, reachN)

## Для media

Доступные атрибуты и допустимые для них значения можно получить методом `get_media_unit()`:

In [ ]:
cats.get_media_unit()

## Для media total

Доступные атрибуты и допустимые для них значения можно получить методом `get_media_total_unit()`:

In [ ]:
cats.get_media_total_unit()

## Для duplication media

Доступные атрибуты и допустимые для них значения можно получить методом `get_media_duplication_unit()`:

In [ ]:
cats.get_media_duplication_unit()

## Для рекламы

Доступные атрибуты и допустимые для них значения можно получить методом `get_ad_unit()`:

In [ ]:
cats.get_ad_unit()

## Для duplication рекламы

Доступные атрибуты и допустимые для них значения можно получить методом `get_media_duplication_unit()`:

In [ ]:
cats.get_profile_duplication_unit()

## Для media & рекламы

Доступные атрибуты и допустимые для них значения можно получить методом `get_media_profile_unit()`:

In [ ]:
cats.get_media_profile_unit()

# Доступный период данных

Узнать доступный период данных можно после выполнения ячейки ниже

In [ ]:
cats.get_date_range()

# Типы пользования интернетом
Получить список типов пользования интернетом (usetype) отдельно для media и рекламы можно после выполнения ячейки ниже

In [ ]:
cats.get_usetype()

Получить список типов пользования интернетом (usetype) для media & рекламы можно после выполнения ячеек ниже.

Метод `get_media_usetype()`- для media:

In [ ]:
cats.get_media_usetype()

Метод `get_profile_usetype()`- для рекламы:

In [ ]:
cats.get_profile_usetype()

# Социально-демографические и географические переменные

Получим список всех переменных, доступных в проекте Cross Web

In [ ]:
df_demo = cats.get_property()
df_demo

Экспорт справочника в Excel. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [ ]:
demo_cat = cats.get_property()

with pd.ExcelWriter('demo_cat.xlsx') as writer:
    demo_cat.to_excel(writer, sheet_name='demo_cat', index=False)

### Важно!
Demo и geo переменные, которые используются в заданиях, должны быть прописаны строго в следующем формате: 

__demoVariable__

То есть первое слово в переменной должно быть написано со строчной буквы, второе - с заглавной. Пример: __ageGroup__

### Поиск переменных

Поиск переменных осуществляется с помощью функции `cats.find_property()`, которая находит все переменные, содержащие заданный текст как в самом ее названии, так и в ее названии для API Cross Web (поля 'name' и 'entityTitle'). 

Если нужно найти переменную без входящих в нее категорий, то необходимо передать в эту функцию дополнительный параметр

```
    expand=False
```

Если нужно найти переменную с входящими в нее категориями, то можно не передавать параметр expand. В таком случае для него автоматически проставится значение True.

Найдем переменные, которые содержат текст "Пол"

In [ ]:
cats.find_property('Пол')

Рассмотрим географические переменные.

Численность населенного пункта (cityPop) и численность населенного пункта 100k+ или 100k- (cityPop100)

In [ ]:
cats.find_property('cityPop', expand=True)

Федеральный округ (federalOkrug)

In [ ]:
cats.find_property('federalOkrug', expand=True)

# Медиа справочник

Получим весь медиа справочник и отобразим первые пять записей.

### Важно!

**Загрузка всего медиа справочника может потребовать много памяти и времени. Для экономии памяти и ускорения загрузки, пожалуйста, пользуйтесь поиском по медиа справочнику.**

In [ ]:
df_media = cats.get_media()
df_media.head()

Экспорт медиа справочника в Excel. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [ ]:
media_cat = cats.get_media()

with pd.ExcelWriter('media_cat.xlsx') as writer:
    media_cat.to_excel(writer, sheet_name='media_cat', index=False)

**В Cross Web трехуровневая иерархия медиа объектов:**

- **Холдинг (holding)** = Холдинг
- **Ресурс (resource)** = Проект/Сайт
- **Медиа продукт (product)** = Сабсекция

**Холдинг** объединяет ресурсы, которые юридически принадлежат одной компании. 

**Ресурс** объединяет в себе веб аудиторию (десктопный, мобильный веб) и приложения. 

**Медиа продукт** - это раздел в десктопном и мобильном вебе или приложение. Разделы в десктопном и мобильном вебе определяются регулярным выражением.

**В Cross Web тематики определяются двумя способами:**

**- тематики для медиа продуктов (theme)**

Таким образом, разные разделы одного ресурса могут относиться к разным тематикам.

**- тематики для ресурсов (resourceTheme)**

В этом случае для всего ресурса выбирается только одна наиболее подходящая тематика, даже если ресурс состоит из медиа продуктов, относящихся к различным тематикам.

### Холдинги
Получим список холдингов

In [ ]:
df_holdings = cats.get_holding()
df_holdings

### Ресурсы
Получим список ресурсов и отобразим первые пять записей

In [ ]:
df_resources = cats.get_resource()
df_resources.head()

### Медиа продукты
Получим список медиа продуктов и отобразим первые пять записей

In [ ]:
df_products = cats.get_product()
df_products.head()

### Тематики для медиа продуктов
Получим список тематик для медиа продуктов и отобразим первые пять записей

In [ ]:
df_thematics = cats.get_theme()
df_thematics.head()

### Тематики для ресурсов
Получим список тематик для ресурсов и отобразим первые пять записей

In [ ]:
df_resource_thematics = cats.get_resource_theme()
df_resource_thematics.head()

### Поиск в медиа справочнике

Поиск в медиа справочнике осуществляется с помощью следующих методов: 

- **get_media()** - отображает все поля медиа справочника; 
- **get_holding()** - отображает название холдинга и его id; 
- **get_resource()** - отображает название ресурса и его id; 
- **get_product()** - отображает название медиа продукта и его id; 
- **get_theme()** - отображает название тематики для медиа продуктов и ее id;
- **get_resource_theme()** - отображает название тематики для ресурсов и ее id.

Для всех перечисленных выше методов поддерживается поиск по следующим параметрам:

- **holding** - поиск по названию холдинга. Возможно задавать часть названия.
- **holding_ids** - поиск по cписку идентификаторов холдингов.
- **resource** - поиск по названию ресурса. Возможно задавать часть названия.
- **resource_ids** - поиск по cписку идентификаторов ресурсов.
- **product** - поиск по названию медиа продукта. Возможно задавать часть названия.
- **product_ids** - поиск по cписку идентификаторов медиа продуктов.
- **theme** - поиск по названию тематики для медиа продуктов. Возможно задавать часть названия.
- **theme_ids** - поиск по cписку идентификаторов тематик для медиа продуктов.
- **resource_theme** - поиск по названию тематики для ресурсов. Возможно задавать часть названия.
- **resource_theme_ids** - поиск по cписку идентификаторов тематик для ресурсов.


#### Поиск по названию
Найдем холдинг "Авито" в медиа справочнике и выведем всю информацию по его медиа объектам

In [ ]:
df_avito = cats.get_media(holding='Авито')
df_avito

#### Поиск по идентификатору
Найдем все медиа объекты с идентификаторами тематик для медиа продуктов
- id 20 - Почта
- id 42 - Работа

In [ ]:
df_media_by_id = cats.get_media(theme_ids=[20, 42])
df_media_by_id

#### Поиск по нескольким параметрам

Параметры поиска можно комбинировать. 

Добавим к предыдущему примеру еще один параметр поиска - название ресурса Avito. 

In [ ]:
df_media_by_id_resource = cats.get_media(theme_ids=[20, 42], resource='Avito')
df_media_by_id_resource

# Справочник для рекламы

**В Cross Web следующая иерархия для рекламы:**

- Agency (рекламное агентство)
- Brand (бренд)
- Campaign (рекламная кампания)
- Advertisement (рекламная позиция)

### Рекламные агентства

Получим список рекламных агентств и отобразим первые пять записей

In [ ]:
df_ad_agency = cats.get_ad_agency()
df_ad_agency.head()

### Бренды

Получим список брендов и отобразим первые пять записей

In [ ]:
df_brand = cats.get_brand()
df_brand.head()

### Рекламные кампании

Получим список рекламных кампаний и отобразим первые пять записей

In [ ]:
df_ad_campaign = cats.get_ad_campaign()
df_ad_campaign.head()

### Поиск в справочнике по рекламе

Поиск в справочнике по рекламе осуществляется с помощью следующих методов: 

- **get_ad()** - отображает все поля справочника по рекламе; 
- **get_ad_list()** - отображает все поля справочника по списку реклам;
- **get_ad_agency()** - отображает название рекламного агентства и его id; 
- **get_brand()** - отображает название бренда и его id; 
- **get_ad_campaign()** - отображает название рекламной кампании и ее id.

Для всех перечисленных выше методов поддерживается поиск по следующим параметрам без учета регистра:

- **agency** - поиск по названию рекламного агентства. Возможно задавать часть названия.
- **agency_ids** - поиск по cписку идентификаторов рекламных агентств.
- **brand** - поиск по названию бренда. Возможно задавать часть названия.
- **brand_ids** - поиск по cписку идентификаторов брендов.
- **campaign** - поиск по названию рекламной кампании. Возможно задавать часть названия.
- **campaign_ids** - поиск по cписку идентификаторов рекламных кампаний.
- **ad** - поиск по названию рекламной позиции. Возможно задавать часть названия.
- **ad_ids** - поиск по cписку идентификаторов рекламных позиций.
- **tmsec_msid** - поиск по тематической метке контента.
- **advertisement_type** - поиск по типу рекламы.
- **advertisement_description** - поиск по кастомному описанию рекламы.
- **created_date** - поиск по дате создания счетчика.
- **start_date** - поиск по дате старта срабатывания счетчика.
- **end_date** - поиск по дате окончания срабатывания счетчика.


Параметры поиска можно комбинировать. 

#### Поиск по названию
Найдем определенное рекламное агентство в справочнике по рекламе и выведем все его поля.

Вместо 'XXXXX' необходимо ввести хотя бы часть названия искомого рекламного агентства.

In [ ]:
df_find_agency = cats.get_ad(agency='XXXXX')
df_find_agency

#### Поиск по идентификатору
Найдем рекламные кампании с идентификаторами брендов 

- id XX - Бренд 1
- id YY - Бренд 2

Вместо 'XX' и 'YY' необходимо ввести идентификаторы брендов (id).

In [ ]:
df_find_campaign = cats.get_ad_campaign(brand_ids=[XX, YY])
df_find_campaign 

#### Поиск по рекламным полям
Найдем записи в дереве рекламы, соответствующие условиям поиска 

- agency XX - Имя агенства
- tmsec_msid YY - Тематическая метка контента
- advertisement_type - Тип рекламы
- created_date - Дата создания счетчика
- start_date - Дата старта срабатывания счетчика
- end_date - Дата окончания срабатывания счетчика

In [ ]:
cats.get_ad(
    agency='XX', 
    tmsec_msid='YY',
    #advertisement_type='mediascope',
    #created_date='2020-10-19', 
    #start_date='2020-10-19',
    end_date='2020-12-08',
    use_cache=False #для того, чтобы искать не в кеше(по умолчанию), а запрашивать свежий результат с сайта
)